In [23]:
# instllation
# pip install celltypist



import os
import pandas as pd
import numpy as np
import celltypist
from celltypist import models
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
workdir = "/Users/funingtian/Dropbox/UChicago/CRI_ftian1/scRNAseq/"
os.chdir(workdir)

In [ ]:
# models.download_models(force_update = True)
model = models.Model.load(model = 'Human_Lung_Atlas.pkl')
count_matrix = "integration_2_leiden/metadata/integration_2_leiden_rna_counts.csv"
predictions = celltypist.annotate(count_matrix, model = model, transpose_input = True)

In [ ]:
predictions.predicted_labels[predictions.predicted_labels['predicted_labels'].isna()]

In [ ]:
predictions.to_table(folder = 'integration_2_leiden/metadata/', 
                     prefix = 'integration_2_leiden_celltypist_')

In [97]:
metadata = pd.read_csv('integration_2_leiden/metadata/integration_2_leiden_meta.data.csv')
original = pd.read_csv('integration_2_leiden/metadata/integration_1_louvain_metadata.csv')
pred = pd.read_csv("integration_2_leiden/metadata/integration_2_leiden_celltypist_predicted_labels.csv")

In [99]:
# metadata.head()

In [101]:
# metadata[metadata['orig.ident'] == "681_Ig_18h"].head()

In [103]:
# substring that last underscore in both metadata and original_681_Ig_18h
metadata.loc[:, 'new_barcodes'] = metadata['Unnamed: 0'].apply(lambda x: '_'.join(x.split('_')[:-1]))
original.loc[:, 'new_barcodes'] = original['Unnamed: 0'].apply(lambda x: '_'.join(x.split('_')[:-1]))

metadata['expCond.media'] = 'CTS' #!!!!!
# create a mapping of 'new_barcodes' to 'orig.ident' from original_681_Ig_18h
for col in ['expCond.media', 'orig.ident']: # do media first, otherwise orig.ident changes
    barcode_681_Ig_18h = original[original['orig.ident.type'] == "681_Ig_18h"].set_index('new_barcodes')[col].to_dict()
    barcode_other = original[original['orig.ident.type'] != "681_Ig_18h"].set_index('new_barcodes')[col].to_dict()
    metadata.loc[
        metadata['orig.ident'] == "681_Ig_18h", col
    ] = metadata.loc[
        metadata['orig.ident'] == '681_Ig_18h', 'new_barcodes'
    ].map(barcode_681_Ig_18h).fillna(metadata[col]) # Update 'orig.ident' only for '681_Ig_18h'
    metadata.loc[
        metadata['orig.ident'] == "original", col
    ] = metadata.loc[
        metadata['orig.ident'] == "original", 'new_barcodes'
    ].map(barcode_other).fillna(metadata[col])     # Update 'orig.ident' for other conditions


# Drop the temporary 'new_barcodes' column
metadata = metadata.drop(columns=['new_barcodes'])

In [105]:
# after removing the last _, no duplicates in either category
original[original['orig.ident.type'] == "681_Ig_18h"]["new_barcodes"].duplicated().any()

False

In [107]:
original[original['orig.ident.type'] != "681_Ig_18h"]["new_barcodes"].duplicated().any()

False

In [109]:
# populate other columns
metadata.loc[:, 'expCond.sample'] = metadata['orig.ident'].apply(lambda x: '_'.join(x.split('_')[1:]))
metadata['expCond.sample'] = metadata['expCond.sample'].str.replace('unstim', 'Unstim', regex=True)
metadata['expCond.sample'] = metadata['expCond.sample'].str.replace('RPMI', '0h', regex=True)
metadata['expCond.sample'] = metadata['expCond.sample'].str.replace('CTS', '0h', regex=True)
metadata.loc[:, 'expCond.donor'] = metadata['orig.ident'].apply(lambda x: x.split('_')[1])

metadata.loc[:, 'expCond.time'] = metadata['orig.ident'].apply(lambda x: x.split('_')[-1])
metadata['expCond.time'] = metadata['expCond.time'].str.replace('RPMI', '0h', regex=True)
metadata['expCond.time'] = metadata['expCond.time'].str.replace('CTS', '0h', regex=True)
metadata.loc[:, 'expCond.lib'] = metadata['orig.ident'].apply(lambda x: x.split('_')[0])
metadata.loc[:, 'expCond.stimuli'] = metadata['orig.ident'].apply(lambda x: x.split('_')[-2])

metadata['expCond.stimuli.time'] = metadata['expCond.stimuli'] + "_" + metadata['expCond.time']
metadata['expCond.donor.stimuli.time'] = metadata['expCond.donor'].astype(str) + "_" + metadata['expCond.stimuli.time']

asthma = {'569': "Asthmatic", '589': "Asthmatic", '548': "Non-asthmatic", 
 '699': "Non-asthmatic", '678': "Asthmatic", '633': "Non-asthmatic", 
 '640': "Non-asthmatic", '662': "Asthmatic", '616': "Non-asthmatic",
 '602': "Asthmatic", '681': "Non-asthmatic", '673': "Asthmatic"}

metadata['expCond.asthma'] = metadata['expCond.donor'].map(asthma)

metadata = metadata.drop(columns=['expCond', 'expCond1'])

In [ ]:
metadata_annot = pd.merge(metadata, pred, on=['Unnamed: 0'], how='left') 

In [ ]:
metadata_annot.isna().any().any()

In [ ]:
metadata_annot.to_csv("integration_2_leiden/metadata/integration_2_leiden_meta.data.celltypist.csv", 
                      index=False)